# Example script for V1 ring model
Model found in

Dynamic signal tracking in a simple V1 spiking model, Guillaume Lajoie, Lai-Sang Young, Neural Computations, (2016), Vol. 28, No. 9, Pages 1985-2010, DOI:10.1162/NECO_a_00868

REQUIREMENTS(files must be in path):
'V1_EIF_architecture_maker.py'
'V1_EIF_NETWORK.py'

The notebook below simulates a network of exponential integrate-and-fire (EIF) neurons arranged in a ring representing their orientation tuning preference. Connectivity between neurons also depends on tuning. There are both excitatory and inhibitory neurons. An external signal representing an oriented edge drives activity in the network.

The script below is a simple example using a rotating signal with increasing strength. It is divided in three parts:

(1) Declare run parameters and build network architecture

(2) Run numerical integrator for simulation

(3) Plot some resutls


In [1]:
##
#Build simulation parameter dictionary and create network 
#architecture 
##

#Module imports
import numpy as np
pi=np.pi

#Function imports (source files must be in path)
from V1_EIF_architecture_maker import EIF_architecture


#-------------------------------
#PARAMETERS (building dictionary p)
#-------------------------------
#Run details
p={
'run_name':'example_run',
'output_dir':'./output/',
'N':1000, #network size
'E_frac':0.75 #proportion of excitatory cells
}

#Switches deciding what to save
p.update({
'traj_switch':False,#Save neural trajectories in voltage space
'synaptic_switch':False,#Save each synaptic currents
'spike_switch':True,#Save spike times
'input_switch':False#Save external noise neurons are getting
})

#Solver parameters
p.update({
't_span':1000.,#simulation length (ms)
'dt':0.1, #is best at 0.001
't_record':0.5, #save traj and/or inputs every t_record
'N_rec_index':range(p['N']), #index of cells to record from
'num_IC':20, #number of random trials  
})

#################
#SIGNAL TYPE HERE
#################
#Oriented signal parameters (gradual rotation at each switch time)
or_dt=100#time resolution of signal
or_dtht=pi/1000
p['or_tuning']=pi/10.#width of signal in orientation sopace
p['or_switch_times']=[(i+1)*or_dt for i in range(int(p['t_span']/or_dt))]
p['or_values']=[or_dtht*i for i in p['or_switch_times']]
p['or_strengths']=list(np.linspace(0.2,1.,len(p['or_switch_times'])))
#################
#################

#Neural dynamics parameters
p.update({
'C':1.,#microF/cm^2
'gL':0.1,#mS/cm^2
'DT':3.48,#mV
'VT':-59.9,#mV
'EL':-65.,#mV
'tref':1.7,#msec (refractory period)
'Vth':-30., #hybrid spike cutoff
'Vr':-68.,#mV
'IC':np.random.uniform(-75,-50,p['N'])#initial conditions for all neurons    
})

#Synaptic delays
#CAUTION: here 'XY' means X-->Y connection !!!!!!!
p.update({
'EE_delay':3.5,
'IE_delay':2.,
'EI_delay':2.,
'II_delay':2.,   
})

#Synaptic connectivity
#CAUTION: here 'XY' means X-->Y connection !!!!!!!
connectivity_weigths={
'EE':1.0,
'EI':0.85,
'IE':-0.75,
'II':-0.85,
'I_NMDA_frac':0.5,
'E_NMDA_frac':0.25
}
p['connectivity_weigths']=connectivity_weigths

#Connectivity width in angular space
#CAUTION: here 'XY' means X-->Y connection !!!!!!!
connectivity_sigmas={
'EE':pi/6.,
'EI':pi/6.,
'IE':pi/10.,
'II':pi/10.
}
p['connectivity_sigmas']=connectivity_sigmas

#Connectivity probability scaling
#CAUTION: here 'XY' means X-->Y connection !!!!!!!
connectivity_prob={
'EE':0.15,
'EI':.5,
'IE':.5,
'II':.5
}
p['connectivity_prob']=connectivity_prob

#Synaptic time constants
#CAUTION: here 'XY' means X-->Y connection !!!!!!!
time_constants={
'EE':2.,
'EI':2.,
'IE':7.,
'II':7.,
'NMDA_EE':100.,
'NMDA_EI':100.
}
p['time_constants']=time_constants

#External sigmoid input scaling
ext_w_p={
'E_center':0.5,
'E_slope':50.,
'E_baseline':.9,
'I_center':0.66,
'I_slope':50.,
'I_baseline':0.9
}
p['ext_w_p']=ext_w_p #saving in main param

#Background noise parameters
p['I']=0.2*np.ones(p['N'])#offset constant current 
p['eps']=1.*np.ones(p['N'])#noise amplitude
#-------------------------------
#END of PARAMETERS (building dictionary p)
#-------------------------------

#-------------------------------
#BUILDING NETWORK ARCHITECTURE
#-------------------------------
#Calling architecture making function
W,Tau,E_index,I_index,id_dict,post_indices,E_post_indices,I_post_indices,w_list,wE_list,wI_list,tau_array,NMDA_frac_dict=EIF_architecture(p['N'],p['E_frac'],connectivity_sigmas,connectivity_prob,connectivity_weigths,time_constants)

#Threshold spiking current
p['IT']=p['gL']*(p['VT']-p['EL'])-p['gL']*p['DT']

#Generating weights for external inputs
E_temp=1/(1+np.exp(-ext_w_p['E_slope']*(np.random.uniform(0,1,len(E_index))-ext_w_p['E_center'])))
E_temp=E_temp+(1.-E_temp)*ext_w_p['E_baseline']
I_temp=1/(1+np.exp(-ext_w_p['I_slope']*(np.random.uniform(0,1,len(I_index))-ext_w_p['I_center'])))
I_temp=I_temp+(1.-I_temp)*ext_w_p['I_baseline']
temp=np.ndarray(p['N'])
E_n=0
I_n=0
for it in range(p['N']):
	if it in E_index:
		temp[it]=E_temp[E_n]
		E_n+=1
	else:
		temp[it]=I_temp[I_n]
		I_n+=1
p['ext_w']=temp

#Storing synaptic management references
p['E_index']=E_index
p['I_index']=I_index
p['id_dict']=id_dict
p['post_indices']=post_indices
p['E_post_indices']=E_post_indices
p['I_post_indices']=I_post_indices
p['w_list']=w_list
p['wE_list']=wE_list
p['wI_list']=wI_list
p['tau_array']=tau_array
p['NMDA_frac_dict']=NMDA_frac_dict
#-------------------------------
#END BUILDING NETWORK ARCHITECTURE
#-------------------------------

#Confirmation message
print('-->Network architecture successfully created, ready to simulate.<--')

-->Network architecture successfully created, ready to simulate.<--


In [ ]:
##
#Numerical integration of network dynamics
##

#Module imports
from sys import stdout
import time as tim

#Function imports (source files must be in path)
import V1_EIF_NETWORK as ei

#-------------------------------
#CALLING SOLVER
#-------------------------------
tBegin = tim.mktime(tim.localtime())#start timer
#++++++++++++++++++++++
print('Starting integration...')
out=ei.integrate_EIF(p)#solver call
#++++++++++++++++++++++
tEnd = tim.mktime(tim.localtime())# Stop timer:
comp_time=tim.strftime("H:%H M:%M S:%S",tim.gmtime(tEnd - tBegin))
stdout.write('\n')
print('----->'+p['run_name']+'run is all done !<-----')
print('----->run time: '+comp_time+'<-----')
# #save data
# out['p']=p
# io.savemat(output_dir+'data_'+run_name,out)

Starting integration...
Integrating network dynamics...
Simulation progress: 99 %, about 0.0 seconds remaining
----->example_runrun is all done !<-----
----->run time: H:00 M:00 S:08<-----


In [ ]:
##
#Simple plots of activity
##

#Module imports
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

#signal color plot
ori=out['ori_input']
time=out['time']
plt.pcolor(time[::20],np.linspace(0,pi,p['N']),ori[:,::20])
plt.xlabel('time (ms)')
plt.ylabel('signal orientation (rad)')
plt.title('Orientation signal')
plt.show()

#rater plot of network spikes
spikes=out['spikes']
for n in range(p['N']):
    plt.plot(spikes[n],n*np.ones(len(spikes[n])),'k.')
plt.xlabel('time (ms)')
plt.ylabel('neuron #')
plt.title('Network raster plot (spike times)')
plt.show()

